In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [3]:
import tarfile
import os
mod_path = "all_model"
model_path = "all_model/model.h5"

In [4]:
pip install git+https://github.com/ArnavS1102/all_detection

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Cloning https://github.com/ArnavS1102/all_detection to /tmp/pip-req-build-32c2wrk5
  Running command git clone --filter=blob:none --quiet https://github.com/ArnavS1102/all_detection /tmp/pip-req-build-32c2wrk5
  Resolved https://github.com/ArnavS1102/all_detection to commit 26851affbf08c056a02dac1193f698b3ed0a59da
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from Model.model import PatchEncoder, Patches, recall_m, precision_m, f1_m
import tensorflow as tf

tf.keras.utils.get_custom_objects().update({'Patches': Patches,'PatchEncoder': PatchEncoder,'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
model = tf.keras.models.load_model(model_path)

2023-07-25 17:22:15.080892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 17:22:19.542388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 17:22:19.542657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 168, 168, 3)]        0         []                            
                                                                                                  
 patches (Patches)           (None, None, 768)            0         ['input_1[0][0]']             
                                                                                                  
 patch_encoder (PatchEncode  (None, 100, 16)              13904     ['patches[0][0]']             
 r)                                                                                               
                                                                                                  
 layer_normalization (Layer  (None, 100, 16)              32        ['patch_encoder[0][0]']   

In [7]:
model.save("export/Servo/1")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("export")

INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


In [16]:
s3_response = sagemaker_session.upload_data("model.tar.gz", bucket='artifacts-h5-tflow')#, key_prefix="model")

In [17]:
from sagemaker.tensorflow import TensorFlowModel

bucket_name = 'artifacts-h5-tflow'
sagemaker_model = TensorFlowModel(
    model_data=f"s3://{bucket_name}/data/model.tar.gz",
    role=role,
    framework_version="2.3",
)

In [18]:
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.g4dn.xlarge")

-----!

In [20]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 9.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np
import cv2

def load_and_preprocess_image(image_path, target_size=(168, 168)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image_array = np.expand_dims(image, axis=0)
    return image_array

image_path = 'input_image.jpg'
image_array = load_and_preprocess_image(image_path)

In [26]:
image_array.shape

(1, 168, 168, 3)

In [27]:
response = predictor.predict(image_array)

In [28]:
response

{'predictions': [[-7.84878063, 90.1221771]]}